# Calculating probability of chess moves

For each board score state (Winning, Neutral, Losing) to each end result (Win/Lose/Draw)

Collect move profiles:
- First move
- Number of times piece is moved until result

Final probabilities
- Prob of moving specific piece as first move for each starting state to final result
- Proportion of moves for each piece for each starting state to final result (like an overall strategy)

In [1]:
import os
import json

import numpy as np

### Helper functions

Calculate board score

In [2]:
def calc_score(boardState):
    piece_value = {'p': 1, 'n': 3, 'b': 3, 'r': 5, 'q': 9, 'k': 0}

    black_pieces = [piece["type"] for piece in boardState if piece["color"] == "b"]
    white_pieces = [piece["type"] for piece in boardState if piece["color"] == "w"]

    black_score = sum([piece_value[piece] for piece in black_pieces])
    white_score = sum([piece_value[piece] for piece in white_pieces])

    return black_score, white_score, black_pieces, white_pieces


Parse available pieces

In [3]:
def parse_available_pieces(boardState, isDingBlack):
    if isDingBlack:
        return [piece["type"].capitalize() for piece in boardState if piece["color"] == "b"]
    else:
        return [piece["type"].capitalize() for piece in boardState if piece["color"] == "w"]

Parse Move

In [4]:
def parse_move(move):
    piece_codes = ['K', 'Q', 'B', 'N', "R"]
    if move[0] not in piece_codes:
        move_piece = 'P'
    else:
        move_piece = move[0]
    
    return move_piece

Parse all moves

In [5]:
def parse_all_moves(moves, isDingNextTurn):
    odd_index = moves[::2]
    even_index = moves[1::2]
    if isDingNextTurn:
        return [parse_move(move) for move in odd_index]
    else:
        return [parse_move(move) for move in even_index]

Parse Win

In [6]:
def parse_win(game):
    isDingBlack = game["isDingBlack"]
    result = game["result"]
    if result == "1/2-1/2":
        return "Draw"
    white_result = result[0]
    if (white_result == "1" and not isDingBlack) or (white_result == "0" and isDingBlack):
        return "Win"
    else:
        return "Lose"

Loading one game

In [7]:
def process_one_game(filepath, first_move_winning_state, all_moves_winning_state):
    file = open(filepath)
    game = json.load(file)

    black_score, white_score, black_pieces, white_pieces = calc_score(game["boardState"])
    isDingBlack = game["isDingBlack"]
    nextTurn = game["nextTurn"]
    moves = game["moves"]
    ding_score = black_score if isDingBlack else white_score
    opp_score = white_score if isDingBlack else black_score
    isDingNextTurn = (isDingBlack and nextTurn == 'b') or (not isDingBlack and nextTurn == 'w')

    if ding_score > opp_score:
        ding_position = "Winning"
    elif ding_score == opp_score:
        ding_position = "Neutral"
    else:
        ding_position = "Losing"

    available_pieces = parse_available_pieces(game["boardState"], isDingBlack)


    if isDingNextTurn:
        first_move = parse_move(moves[0])
    else:
        if len(moves) <= 1:
            return
        first_move = parse_move(moves[1])

    all_moves = parse_all_moves(moves, isDingNextTurn)
    num_moves = len(all_moves)
    values, counts = np.unique(all_moves, return_counts=True)
    counts = counts / num_moves
    ding_moves = dict(zip(values, counts.T))


    final_result = parse_win(game)

    first_move_winning_state[ding_position][final_result][first_move] += 1

    for available in available_pieces:
        first_move_winning_state[ding_position][final_result][available + " avail"] += 1

        all_moves_winning_state[ding_position][final_result][available + " avail"] += 1

    for moved in list(ding_moves.keys()):
        all_moves_winning_state[ding_position][final_result][moved] += ding_moves[moved]


Initialise data dictionaries

In [8]:
data_first_move = {"K": 0, "K avail": 0, "Q": 0, "Q avail": 0, "B": 0, "B avail": 0, "N": 0, "N avail": 0, "R": 0, "R avail": 0, "P": 0, "P avail": 0}

first_move_winning_state = {"Winning": {"Win": data_first_move.copy(), "Lose": data_first_move.copy(), "Draw": data_first_move.copy()}, "Neutral": {"Win": data_first_move.copy(), "Lose": data_first_move.copy(), "Draw": data_first_move.copy()}, "Losing": {"Win": data_first_move.copy(), "Lose": data_first_move.copy(), "Draw": data_first_move.copy()}}
all_moves_winning_state = {"Winning": {"Win": data_first_move.copy(), "Lose": data_first_move.copy(), "Draw": data_first_move.copy()}, "Neutral": {"Win": data_first_move.copy(), "Lose": data_first_move.copy(), "Draw": data_first_move.copy()}, "Losing": {"Win": data_first_move.copy(), "Lose": data_first_move.copy(), "Draw": data_first_move.copy()}}

Iterate over all json files in directory

In [9]:
directory_name = "ding_json_5"
directory = os.fsencode(directory_name)
num_json = 0
    
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".json"): 
         num_json +=1
         process_one_game(directory_name + '/' + filename, first_move_winning_state, all_moves_winning_state)
     else:
         continue

print(num_json)

69


In [10]:
states = ["Winning", "Losing", "Neutral"]
results = ["Win", "Lose", "Draw"]

# Normalise all probabilities in each (state, result) to sum to 1
for state in states:
    for result in results:
        all_moves_probabilities = all_moves_winning_state[state][result]
        first_move_probabilities = first_move_winning_state[state][result]
        num_games = all_moves_probabilities['K avail'] # Since there is always a K
        if not num_games:
            continue

        final_all_moves_prob = {key: value/num_games for (key, value) in all_moves_probabilities.items() if len(key) == 1}
        final_first_move_prob = {key: value/num_games for (key, value) in first_move_probabilities.items() if len(key) == 1}

        all_moves_winning_state[state][result] = final_all_moves_prob
        first_move_winning_state[state][result] = final_first_move_prob


### Results

In [11]:
all_moves_winning_state["Losing"]["Draw"]

{'K': 0.5435753640299095,
 'Q': 0.0,
 'B': 0.08333333333333331,
 'N': 0.0,
 'R': 0.23900039354584807,
 'P': 0.1340909090909091}

In [12]:
first_move_winning_state["Losing"]["Draw"]

{'K': 0.5454545454545454,
 'Q': 0.0,
 'B': 0.09090909090909091,
 'N': 0.0,
 'R': 0.3181818181818182,
 'P': 0.045454545454545456}